<a href="https://colab.research.google.com/github/KhushiBhambri/YogNet/blob/main/PoseStreamModel/PoseStreamModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# import sys
# sys.path.append('/content/drive/MyDrive/Major_Project_Cse')

%cd "/content/drive/MyDrive/Major_Project_Cse"

Mounted at /content/drive
/content/drive/MyDrive/Major_Project_Cse


In [30]:
numOfKeypoints = 35
import pandas as pd
import ast

# read the data from the CSV file
data = pd.read_csv('PoseStream_train_data.csv')

# load the keypoints and labels into separate variables
train_keypoints=[]
train_labels = list(data['label'])

for keypoints_str in data['keypoints']:
    keypoints_list = ast.literal_eval(keypoints_str)
    train_keypoints.append(keypoints_list)

In [31]:
# train_keypoints[10]

In [32]:
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import initializers
from keras.layers import Conv3D, MaxPooling3D,Dense,ZeroPadding3D,Flatten,Activation,BatchNormalization,RandomFlip,RandomRotation,Rescaling
from keras.models import load_model


In [33]:
print(len(train_keypoints),len(train_labels))

1213 1213


In [34]:
# calculate the center of the object in each frame
centers = []
for keypoints in train_keypoints:
    frame_centers = []
    for frame_keypoints in keypoints:
        # assume the center is the mean of all keypoints
       center_x = np.nanmean([kp[0] for kp in frame_keypoints if kp[0] is not None])
       center_y = np.nanmean([kp[1] for kp in frame_keypoints if kp[1] is not None])
       frame_centers.append((center_x, center_y))
    centers.append(frame_centers)

train_relative_keypoints = train_keypoints
# make the keypoints relative to the center
for i in range(len(train_relative_keypoints)):
    for j in range(len(train_relative_keypoints[i])):
        for k in range(len(train_relative_keypoints[i][j])):
            if train_relative_keypoints[i][j][k][0] is not None:
               train_relative_keypoints[i][j][k][0] -= centers[i][j][0]
            if train_relative_keypoints[i][j][k][1] is not None:
              train_relative_keypoints[i][j][k][1] -= centers[i][j][1]


<ipython-input-34-a6c48355f099>:7: RuntimeWarning: Mean of empty slice
  center_x = np.nanmean([kp[0] for kp in frame_keypoints if kp[0] is not None])
<ipython-input-34-a6c48355f099>:8: RuntimeWarning: Mean of empty slice
  center_y = np.nanmean([kp[1] for kp in frame_keypoints if kp[1] is not None])


In [35]:
def ReplacebyAverage(index,keypoints):
  
  group_indices = keypoint_groups[index]
  group_keypoints = [keypoints[i] for i in group_indices]
  valid_keypoints = [k for k in group_keypoints if k is not None]
  if len(valid_keypoints) > 0:
     group_average_x = np.mean([k[0] for k in valid_keypoints if k[0] is not None])
     group_average_y = np.mean([k[1] for k in valid_keypoints if k[1] is not None])
    #  keypoints[index][0]=group_average_x
    #  keypoints[index][1]=group_average_y
  else:
    # If all keypoints in the group are missing, replace with (0,0) or another default value
    group_average_x, group_average_y = 0, 0
  print(group_average_x,group_average_y)
  return [group_average_x,group_average_y]

In [54]:
count=0
new_train_keypoints = []
new_train_labels = []
for i in range(len(train_relative_keypoints)):
    # flag=1
    frames_keypoints=[] 
    for j in range(len(train_relative_keypoints[i])):
        flag=1
        for k in range(len(train_relative_keypoints[i][j])):
            if train_relative_keypoints[i][j][k][0] is None or train_relative_keypoints[i][j][k][1] is None:
              count+=1
              flag=0
              break
        if(flag==1):
          frames_keypoints.append(train_relative_keypoints[i][j])
    if(len(frames_keypoints)>0):
      kk = [[-1.0,-1.0] for _ in range(numOfKeypoints)]
      for x in range (16-len(frames_keypoints)):
          frames_keypoints.append(kk)
      # if(len(frames_keypoints)!=16):
      #   print("2 : ",len(frames_keypoints))
      new_train_keypoints.append(frames_keypoints)
      new_train_labels.append(train_labels[i])

In [55]:
print(len(new_train_keypoints),len(new_train_labels))

1031 1031


In [38]:
new_train_keypoints[0]

[[[-0.029326708827699943, -0.026793485028403174],
  [-0.034212203536714836, -0.02457404647554673],
  [-0.0346262173993247, -0.02584773812975205],
  [-0.034937770877565666, -0.027047460419791114],
  [-0.034913213763918205, -0.024447863442557227],
  [-0.035796137366976066, -0.025694852215903174],
  [-0.03663066199847631, -0.026806061608450782],
  [-0.039403291259493156, -0.04115552221025742],
  [-0.04029411247798376, -0.041398470742361915],
  [-0.028009744201387687, -0.03802711282457627],
  [-0.028571100745882316, -0.03803718600954331],
  [-0.03417179158755712, -0.06857920203890122],
  [-0.0349885540349143, -0.07739919934953965],
  [-0.03415009549685888, 0.01500725235257827],
  [-0.03543076089450292, -0.005169873578207862],
  [-0.038290949378694816, 0.10855608667646133],
  [-0.0407183485371726, 0.06685149158750259],
  [-0.04855254462787084, 0.11788504804883682],
  [-0.05034718087741308, 0.07137697424207412],
  [-0.052036555324281974, 0.11120289053235732],
  [-0.049935998235430046, 0.0752

In [56]:
# train_videos = np.asarray(train_videos)
new_train_keypoints = np.asarray(new_train_keypoints)
new_train_labels = np.asarray(new_train_labels).astype('int32')

# train_keypoints = tf.convert_to_tensor(train_keypoints, dtype=tf.float32)
# # yKey_train = tf.cast(yKey_train , dtype=tf.float32)
# train_labels = tf.convert_to_tensor(train_labels, dtype=tf.float32)

# train_keypoints = np.array([np.array(val) for val in train_keypoints])

from sklearn.model_selection import train_test_split

XKey_train,XKey_test,yKey_train,yKey_test = train_test_split(new_train_keypoints,new_train_labels,test_size=0.25,shuffle=True)

In [57]:
print(len(XKey_train),len(XKey_test))

773 258


In [58]:
XKey_train = tf.convert_to_tensor(XKey_train, dtype=tf.float32)
yKey_train = tf.convert_to_tensor(yKey_train, dtype=tf.int32)

In [59]:
# import mediapipe as mp

# landmarks = mp_pose.PoseLandmark.__members__
# print(landmarks)


In [ ]:
from keras import models, layers
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, LSTM, Dense, Dropout, TimeDistributed, GlobalAveragePooling1D
from keras.optimizers import Adam
from keras.initializers import GlorotNormal
from keras.regularizers import L1L2

yKey_train = np.squeeze(yKey_train)

tf.config.run_functions_eagerly(True)

# Define model input shape
input_shape = (16, 35, 2)

# Build model architecture
model = models.Sequential()

model.add(TimeDistributed(Conv1D(filters=16, kernel_size=3, activation='relu'),input_shape = input_shape))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Conv1D(filters=128, kernel_size=3, activation='relu')))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Conv1D(filters=50, kernel_size=3, activation='relu')))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))

# flatten output from CNN layers
model.add(TimeDistributed(GlobalAveragePooling1D()))

# add LSTM layer
# model.add(LSTM(100, dropout=0.0, recurrent_dropout=0.0, kernel_initializer=GlorotNormal(seed=42), recurrent_regularizer=L1L2(l1=0.0, l2=0.001), bias_regularizer=L1L2(l1=0.0, l2=0.001), recurrent_initializer=GlorotNormal(seed=42), return_sequences=False))
model.add(LSTM(100, recurrent_regularizer=L1L2(l1=0.01, l2=0.001), bias_regularizer=L1L2(l1=0.01, l2=0.001), recurrent_initializer=GlorotNormal(seed=42), return_sequences=False))

# TODO: forget bias 
# TODO: L1 = 0.01


# add fully connected layers

# model.add(Dropout(0.5))
# model.add(Dense(100, activation='relu'))

model.add(Dropout(0.5))
model.add(Dense(80, activation='relu'))

# add Softmax layer
model.add(Dense(20, activation='softmax'))

# compile model
adam = Adam(learning_rate=0.001) 
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

# print model summary
# print(model.summary())

history = model.fit(XKey_train,yKey_train,batch_size=32,epochs = 400,validation_split = 0.2)

Epoch 1/400
20/20 [==============================] - 3s 136ms/step - loss: 21.7000 - accuracy: 0.0615 - val_loss: 18.0037 - val_accuracy: 0.0387
Epoch 2/400
20/20 [==============================] - 3s 129ms/step - loss: 15.4152 - accuracy: 0.0534 - val_loss: 12.6308 - val_accuracy: 0.0516
Epoch 3/400
20/20 [==============================] - 3s 132ms/step - loss: 10.7310 - accuracy: 0.1100 - val_loss: 8.6997 - val_accuracy: 0.1161
Epoch 4/400
20/20 [==============================] - 3s 165ms/step - loss: 7.3103 - accuracy: 0.1521 - val_loss: 5.8877 - val_accuracy: 0.3097
Epoch 5/400
20/20 [==============================] - 3s 135ms/step - loss: 5.0454 - accuracy: 0.2298 - val_loss: 4.1440 - val_accuracy: 0.3806
Epoch 6/400
20/20 [==============================] - 3s 128ms/step - loss: 3.6963 - accuracy: 0.3042 - val_loss: 3.2563 - val_accuracy: 0.3097
Epoch 7/400
20/20 [==============================] - 3s 129ms/step - loss: 2.9551 - accuracy: 0.3641 - val_loss: 2.7889 - val_accuracy: 0

In [ ]:


# Get model summary
# model.summary()

# # Get training and validation accuracy
# train_accuracy = history.history['accuracy']
# val_accuracy = history.history['val_accuracy']

# # Get training and validation loss
# train_loss = history.history['loss']
# val_loss = history.history['val_loss']


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_7 (TimeDis  (None, 16, 32, 16)       112       
 tributed)                                                       
                                                                 
 time_distributed_8 (TimeDis  (None, 16, 16, 16)       0         
 tributed)                                                       
                                                                 
 time_distributed_9 (TimeDis  (None, 16, 14, 128)      6272      
 tributed)                                                       
                                                                 
 time_distributed_10 (TimeDi  (None, 16, 7, 128)       0         
 stributed)                                                      
                                                                 
 time_distributed_11 (TimeDi  (None, 16, 5, 50)       

In [52]:
model.evaluate(XKey_test,yKey_test)

9/9 [==============================] - 0s 53ms/step - loss: 0.5969 - accuracy: 0.9070


[0.5968587398529053, 0.9069767594337463]

In [53]:
# Save the model to a file
model.save('PoseStreamFrame.h5')

# Load the saved model from the file
loaded_model = load_model('PoseStreamFrame.h5')

In [51]:
loaded_model.evaluate(XKey_test,yKey_test)

9/9 [==============================] - 1s 65ms/step - loss: 0.7392 - accuracy: 0.8643


[0.739200234413147, 0.8643410801887512]

In [ ]:
input_data = new_train_keypoints[0]
# Reshape the input data to include the batch size dimension
input_data = np.expand_dims(input_data, axis=0)

# Make a prediction with the model
output = model.predict(input_data)
output

/usr/local/lib/python3.9/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


1/1 [==============================] - 0s 299ms/step


array([[9.99708712e-01, 2.62607158e-11, 8.06678546e-09, 1.93838652e-08,
        4.37154535e-09, 3.91757027e-07, 7.37984745e-19, 7.65237303e-07,
        2.77937128e-04, 2.42370291e-11, 1.20226105e-05, 3.20784697e-12,
        1.26892900e-07, 1.34451783e-09, 6.71775569e-13, 3.91274763e-10,
        7.01205912e-13, 3.00378666e-10, 9.27593319e-15, 4.94650870e-13]],
      dtype=float32)

In [49]:
history.history

{'loss': [21.697357177734375,
  15.334752082824707,
  10.374938011169434,
  6.945734977722168,
  4.801348686218262,
  3.59242844581604,
  2.8934285640716553,
  2.7839713096618652,
  2.6048779487609863,
  2.5306413173675537,
  2.5051467418670654,
  2.4700725078582764,
  2.3183608055114746,
  2.2186343669891357,
  2.219428062438965,
  2.204704523086548,
  2.1441214084625244,
  2.101477861404419,
  2.064624547958374,
  2.0200462341308594,
  1.9735335111618042,
  1.9881101846694946,
  1.8807616233825684,
  1.8255115747451782,
  1.796167016029358,
  1.7502628564834595,
  1.8021289110183716,
  1.7942311763763428,
  1.7058643102645874,
  1.6513512134552002,
  1.6281332969665527,
  1.5865105390548706,
  1.5486098527908325,
  1.5636802911758423,
  1.539646863937378,
  1.5715489387512207,
  1.4929057359695435,
  1.4360692501068115,
  1.3862661123275757,
  1.3283048868179321,
  1.2949881553649902,
  1.3075218200683594,
  1.1945618391036987,
  1.2245590686798096,
  1.1564655303955078,
  1.16013193

In [50]:
history = model.fit(XKey_train,yKey_train,batch_size=32,epochs = 100,validation_split = 0.2)

Epoch 1/100
 1/20 [>.............................] - ETA: 2s - loss: 0.3322 - accuracy: 0.8750

/usr/local/lib/python3.9/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


20/20 [==============================] - 3s 150ms/step - loss: 0.3438 - accuracy: 0.9094 - val_loss: 0.7776 - val_accuracy: 0.8645
Epoch 2/100
20/20 [==============================] - 3s 152ms/step - loss: 0.3549 - accuracy: 0.9175 - val_loss: 0.7804 - val_accuracy: 0.8645
Epoch 3/100
20/20 [==============================] - 3s 154ms/step - loss: 0.2679 - accuracy: 0.9466 - val_loss: 0.6993 - val_accuracy: 0.8774
Epoch 4/100
20/20 [==============================] - 3s 127ms/step - loss: 0.2401 - accuracy: 0.9401 - val_loss: 0.6524 - val_accuracy: 0.8903
Epoch 5/100
20/20 [==============================] - 3s 129ms/step - loss: 0.2298 - accuracy: 0.9434 - val_loss: 0.6560 - val_accuracy: 0.9097
Epoch 6/100
20/20 [==============================] - 3s 130ms/step - loss: 0.2305 - accuracy: 0.9450 - val_loss: 0.6870 - val_accuracy: 0.8774
Epoch 7/100
20/20 [==============================] - 3s 170ms/step - loss: 0.2248 - accuracy: 0.9337 - val_loss: 0.6724 - val_accuracy: 0.8968
Epoch 8/100